In [1]:
from apiKey import lastFMKey
from apiKey import lastFMSecret

#import needed libraries
import pandas as pd
import matplotlib as plt
import json
import numpy
import requests as req
import pylast
import time
import iso3166 as iso
import csv

# print(lastFMKey)
# print(lastFMSecret)

#### Declare vairables

In [2]:
countryCsv = "../Output/countries.csv"
counrtyTopArtistCSV = "../Output/countriesTopArtist.csv"
countriesArtistNotFound = "../Output/countriesArtistNotFound.txt"
artistCSV = "../Output/artist.csv"
jsonFile = "../Output/lastFMCountryArtist.json"


#### Def methods for calling and displaying

In [3]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': 'bootcamp'}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = lastFMKey
    payload['format'] = 'json'

    response = req.get(url, headers=headers, params=payload)
    return response

In [4]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

#### Country collection and export to csv

In [5]:
params = open(countryCsv, "w")
countries = iso.countries_by_name
countries.pop('UNITED STATES OF AMERICA', None)
countries["UNITED STATES"] = ("UNITED STATES")
countries["UNITED KINGDOM"] = ("UNITED KINGDOM")
#countries

csvwriter = csv.writer(params)

for country in countries.items():
    csvwriter.writerow(country)

#### Loop through countries and collect top 50 per country

#### Get list of top artist per country and add to dataframe

In [6]:
iterator = 0
header = True
mode = 'w'

for country in countries:
    
    iterator += 1
    if (iterator > 1):
        header = False
        mode = 'a'
        
    responseTopArt = lastfm_get({
        'method':'geo.getTopArtists',
        'country':country,
        'limit':'50'
    })
    r = responseTopArt.json()
    
    try:
        #print(f"Looking to top artists for {country}.  Mode: {mode}. Iterator:{iterator}")
        artistDF = pd.json_normalize(r,[["topartists","artist"]])
        artistDF['country'] = country
              
        with open(counrtyTopArtistCSV, mode) as f:
            artistDF.to_csv(f, header=header)
            
    except:
        print(f"No artists found for {country}")
    #throttle api calls by 1 sec.  
    time.sleep(1)
    
# jprint(r.json())

No artists found for BOLIVIA, PLURINATIONAL STATE OF
No artists found for BONAIRE, SINT EUSTATIUS AND SABA
No artists found for CABO VERDE
No artists found for CONGO, DEMOCRATIC REPUBLIC OF THE
No artists found for CÔTE D'IVOIRE
No artists found for CZECHIA
No artists found for HOLY SEE
No artists found for KOSOVO
No artists found for LIBYA
No artists found for NORTH MACEDONIA
No artists found for MOLDOVA, REPUBLIC OF
No artists found for PALESTINE, STATE OF
No artists found for RÉUNION
No artists found for SAINT HELENA, ASCENSION AND TRISTAN DA CUNHA
No artists found for SINT MAARTEN (DUTCH PART)
No artists found for ESWATINI
No artists found for TAIWAN, PROVINCE OF CHINA
No artists found for UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND
No artists found for VENEZUELA, BOLIVARIAN REPUBLIC OF


#### Get Artist information and save to CSV.

In [ ]:
iterator = 0
header = True
mode = 'w'

artistDF = pd.read_csv(counrtyTopArtistCSV)
artistDedupeDF = artistDF['name'].drop_duplicates()

for artist in artistDedupeDF:
    
    iterator += 1
    if (iterator > 1):
        header = False
        mode = 'a'
        
    responseArtist = lastfm_get({
        'method':'artist.getInfo',
        'artist': artist
    })

    a = responseArtist.json()
    

    print(f"Looking to top artists for {artist}.  Mode: {mode}. Iterator:{iterator}")

    artistDF = pd.json_normalize(a)

    finalArtDF = artistDF[["artist.name","artist.stats.listeners","artist.stats.playcount","artist.streamable","artist.tags.tag"]].head()
    
    with open(artistCSV, mode) as f:
        finalArtDF.to_csv(f, header=header)

# # data = json.loads()
# jprint(r.json())

Looking to top artists for The xx.  Mode: w. Iterator:1
Looking to top artists for The Beatles.  Mode: a. Iterator:2
Looking to top artists for Radiohead.  Mode: a. Iterator:3
Looking to top artists for Lana Del Rey.  Mode: a. Iterator:4
Looking to top artists for Pink Floyd.  Mode: a. Iterator:5
Looking to top artists for Metallica.  Mode: a. Iterator:6
Looking to top artists for Drake.  Mode: a. Iterator:7
Looking to top artists for Nirvana.  Mode: a. Iterator:8
Looking to top artists for Katy Perry.  Mode: a. Iterator:9
Looking to top artists for Kendrick Lamar.  Mode: a. Iterator:10
Looking to top artists for Led Zeppelin.  Mode: a. Iterator:11
Looking to top artists for blink-182.  Mode: a. Iterator:12
Looking to top artists for Depeche Mode.  Mode: a. Iterator:13
Looking to top artists for The Rolling Stones.  Mode: a. Iterator:14
Looking to top artists for Oasis.  Mode: a. Iterator:15
Looking to top artists for Michael Jackson.  Mode: a. Iterator:16
Looking to top artists for Ma

In [45]:
# responseArtist = lastfm_get({
#     'method':'artist.getInfo',
#     'artist':"Metallica"
# })

#a = responseArtist.json()

artistDF = pd.json_normalize(a)#,[["artist","stats"]])

artistDF[["artist.name","artist.stats.listeners","artist.stats.playcount","artist.streamable","artist.tags.tag"]].head()
# artist.stats.listeners

jprint(a)

{
    "artist": {
        "bio": {
            "content": "Metallica is an American heavy metal band. The band was formed in 1981 in San Francisco by vocalist/guitarist James Hetfield and drummer Lars Ulrich and has been based in San Francisco for most of its career. The band's fast tempos, instrumentals, and aggressive musicianship made them one of the founding \"big four\" bands of thrash metal, alongside Megadeth, Anthrax, and Slayer. Metallica's current lineup comprises founding members, primary songwriters Hetfield, Ulrich, longtime lead guitarist Kirk Hammett, and bassist Robert Trujillo. Guitarist Dave Mustaine (who went on to form Megadeth after being fired from the band) and bassists Ron McGovney, Cliff Burton (who died in a bus accident in Sweden in 1986) and Jason Newsted are former members of the band.\n\nMetallica earned a growing fan base in the underground music community and won critical acclaim with its first five albums. The band's third album, Master of Puppets (1986

In [57]:
# artistDFGenres = pd.json_normalize(a,'tags')#,[["artist","stats"]])
print(a)
#artistDFGenres[["artist.name","artist.tags.tag"]].head()

AttributeError: 'dict' object has no attribute 'json'

In [25]:
artistDF.dtypes

artist.name                    object
artist.mbid                    object
artist.url                     object
artist.image                   object
artist.streamable              object
artist.ontour                  object
artist.stats.listeners         object
artist.stats.playcount         object
artist.similar.artist          object
artist.tags.tag                object
artist.bio.links.link.#text    object
artist.bio.links.link.rel      object
artist.bio.links.link.href     object
artist.bio.published           object
artist.bio.summary             object
artist.bio.content             object
dtype: object

In [9]:
# responseTopArt = lastfm_get({
#     'method':'geo.getTopArtists',
#     'country':'United Kingdom',
#     'limit':'1'
# })
# r = responseTopArt.json()
# jprint(r)